# TAP

This notebook is not intended to be used by the WALLABY science team but provides code snippets for accessing the TAP interface

# Imports

In [ ]:
import os
import requests
import getpass
import pyvo as vo
from astropy.io.votable import parse_single_table

# Select Tag

In [ ]:
URL = "https://wallaby.aussrc.org/tap"
tap = vo.dal.TAPService(URL)

In [ ]:
query = "SELECT * FROM wallaby.tag"
votable = tap.search(query)
table = votable.to_table()

In [ ]:
table

# Input tag, query detections

In [ ]:
tag_name = 'NGC 5044 DR1'

In [ ]:
query = """SELECT d.* FROM wallaby.tag_detection td
        LEFT JOIN wallaby.detection d ON d.id = td.detection_id LEFT JOIN wallaby.tag t ON t.id = td.tag_id
        WHERE t.name IN ('Internal Data Release', '$TAG_NAME')
        GROUP BY d.id"""
query = query.replace('$TAG_NAME', tag_name)

In [ ]:
result = tap.search(query)
table = result.to_table()

In [ ]:
table

# Download products


## Authenticate

In [ ]:
username = 'ashen'
password = getpass.getpass('Enter your password')

## Download

In [ ]:
def download_table_products(table, directory, chunk_size=8192):
    """Download WALLABY products from ADQL queried table

    """
    if not os.path.exists(directory):
        os.mkdir(directory)
    print(f'Saving products to {directory}')
    for row in table:
        name = row['source_name']
        products_filename = os.path.join(directory, f'{name}.tar')
        access_url = row['access_url']
        votable = parse_single_table(access_url)
        product_table = votable.to_table()
        url = product_table[product_table['description'] == 'SoFiA-2 Detection Products'][0]['access_url']
        with requests.get(url, auth=(username, password), stream=True) as r:
            r.raise_for_status()
            with open(products_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=chunk_size):
                    f.write(chunk)
        print(f'Downloaded completed for {name}')

In [ ]:
download_table_products(table[0:5], tag_name)